# Product Matching
## Level 1: Data Transformation Script

### Notes:
1. Preprocess newly fetched data
2. Preliminary data cleaning
3. Engineer data features and columns that will enrich data exploration phase
4. Split data into training and testing subsets

### References:

## Notebook Config

In [4]:
# Display settings
## Auto reload modules & inline plots
%reload_ext autoreload
%autoreload 2
%matplotlib inline

## Package Import & Initialization

In [5]:
# Import packages

import pandas as pd                  #For data manipulation and bgq --> pandas conversion
import numpy as np                   #For scientific computation
import os                            #For work with native operating system and directories
from pathlib import Path             #For working with file paths and directories
import warnings                      #To tweak warning options
import datetime as dt                #For date objects and implemetations
from sklearn.model_selection import train_test_split   #For partitioning data ahead of model development & evaluation

In [6]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format
warnings.filterwarnings('ignore')

## Function Library

## Custom Tranformers

## Set Directories

In [7]:
#cwd = os.getcwd()
#os.chdir(cwd)
home = str(Path.home())
proj_path = os.path.join(home, 'Cardinal Health', 'Enterprise Data Remediation - Documents', 'General', '01_Projects', 
                         '23_Kinaxis')

In [8]:
main_dir = proj_path + '\\'
iput_dir = main_dir + 'Data\\Output\\ML\\'
oput_dir = main_dir + 'Data\\Output\\ML\\'

## Set Output File Names

In [9]:
## Output file name
trn_oput_file_name = "L100_trn_df_" + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.pkl'
tst_oput_file_name = "L100_tst_df_" + str(dt.datetime.today().strftime('%Y-%m-%d')) + '.pkl'

## Load Data

In [10]:
raw_0_df = pd.read_pickle(iput_dir + "L000_fetch_df_2022-07-15.pkl")

In [11]:
raw_1_df = pd.read_pickle(iput_dir + "L001_fetch_df_2022-07-15.pkl")

In [12]:
raw_2_df = pd.read_pickle(iput_dir + "L002_fetch_df_2022-07-15.pkl")

## High-level Data Inspection

In [ ]:
raw_0_df.info()

In [ ]:
raw_0_df.head()

In [ ]:
raw_0_df.Match.value_counts()

In [ ]:
raw_1_df.info()

In [ ]:
raw_1_df.head()

In [ ]:
raw_1_df.Match.value_counts()

In [ ]:
raw_2_df.info()

In [ ]:
raw_2_df.head()

In [ ]:
raw_2_df.Match.value_counts()

## Data Preprocessing

#### Append Data

In [13]:
raw_df = (
    pd.concat([
        raw_0_df,
        raw_1_df,
        raw_2_df.rename(columns = dict(zip(
            ['MATNR','MAKTX','MFRPN','NAME1','ZZM_ITEMTAX4'],
            ['Number','FullDescription','ManufacturerItemNumber','ManufacturerName','UNSPSC']
        )))
    ])
    .reset_index(drop=True)
)

In [14]:
raw_df.Match.unique()

array(['No', 'Yes', nan, '?'], dtype=object)

#### Filter & Shuffle Data

In [15]:
raw_mdl_df = raw_df.query('Match in ["Yes","No"]').sample(frac=1, random_state=42).reset_index(drop=True)

In [16]:
raw_mdl_df.Match.value_counts()

No     1276
Yes     809
Name: Match, dtype: int64

#### First Round of Feature Creation

In [86]:
def f(x):
    if x['tmp_ft_2a']>=x['tmp_ft_2b']:
        ans = x['tmp_ft_1a'][x['tmp_ft_3']:]
    else: 
        ans = x['tmp_ft_1b'][x['tmp_ft_3']:]
    return ans


raw_feat_df = (
    raw_mdl_df
    .assign(
        SUPPLIER_ITEM_NUMBER = lambda df: np.where(df.SUPPLIER_ITEM_NUMBER.isna(), '', df.SUPPLIER_ITEM_NUMBER.astype(str)),
        ManufacturerItemNumber = lambda df: np.where(df.ManufacturerItemNumber.isna(), '', df.ManufacturerItemNumber.astype(str)),
    )
    .assign(
        # tmp_ft_1a = lambda df: df.SUPPLIER_ITEM_NUMBER.str.strip(r'[@#&$%+-_/*]'),
        # tmp_ft_1b = lambda df: df.ManufacturerItemNumber.str.strip(r'[@#&$%+-_/*]'),
        tmp_ft_1a = lambda df: df['SUPPLIER_ITEM_NUMBER'].str.replace(r'[^\w\s]+', ''),
        tmp_ft_1b = lambda df: df['ManufacturerItemNumber'].str.replace(r'[^\w\s]+', ''),
    )
    .assign(
        tmp_ft_2a = lambda df: df['tmp_ft_1a'].str.len(),
        tmp_ft_2b = lambda df: df['tmp_ft_1b'].str.len(),
    )
    .assign(
        tmp_ft_3 = lambda df: 
            np.where(
                df['tmp_ft_2a']>=df['tmp_ft_2b'],
                df['tmp_ft_2a']-df['tmp_ft_2b'],
                df['tmp_ft_2b']-df['tmp_ft_2a'],
            ),
    )
    .assign(
        tmp_ft_4 = lambda df: df.apply(f, 1),
    )
    .assign(
        Feat_6 = lambda df: 
            np.where(
                df['tmp_ft_2a']>=df['tmp_ft_2b'],
                np.where(df['tmp_ft_1b']==df['tmp_ft_4'], 1, 0),
                np.where(df['tmp_ft_1a']==df['tmp_ft_4'], 1, 0),
            ),
    )
)

In [89]:
feat_df = raw_feat_df.drop([x for x in raw_feat_df if x.startswith('tmp_ft_')], 1)
feat_df.head(5)

,ITEMNUMBER,Number,ITEMDESCRIPTION,FullDescription,SUPPLIER_ITEM_NUMBER,ManufacturerItemNumber,SUPPLIERNAME,ManufacturerName,UNSPCCode,UNSPSC,Feat_1,Feat_2,Feat_3,Feat_4,Feat_5,Match_Confidence,Match,Comments,Unnamed: 17,Feat_6
0,PYD43600,D43600PDI,Sani Hands ALC Packet with Aloe 5 x8,HAND SANITIZER INSTANT SANI HANDS 100/PK 10PK/CS,D43600,D43600,PDI INC.,"PDI, INC.",53131626.0000,99999999,1,100,50,0,0,01 High,Yes,NaN,NaN,1
1,55AT74635,AT74635,"Surgical Face Mask, Fog-Free, with Eyeshield",MASK SURGICAL EYE SHIELD BL 25/BX 4BX/CS,AT74635,AT74635,CARDINAL HEALTH - MED,CARDINAL HEALTH,42131606.0000,42131713,1,100,56,0,0,01 High,Yes,NaN,NaN,1
2,ZG711NB,ZG711NB,"ReliaMed Tubular Elastic Stretch Net Dressing,...","RELIAMED TUBE NET DRSG, NS,LTX,25YD",711NB,ZG711NB,YASCO ENTERPRISE CORP,ASSURAMED INC,42311543.0000,42311543,0,24,52,1,1,03 Low,Yes,NaN,NaN,1
3,558842,8842,"Cardinal Health Latex Exam Gloves, Non-Sterile...",GLOVES LATEX POSITIVE TOUCH PF 4.7 CREAM MD 10...,8842,8842,CARDINAL HEALTH - MED,CARDINAL HEALTH,42132203.0000,42132203,1,100,51,0,1,01 High,Yes,NaN,NaN,1
4,GL705,GL705,"Surgilast Tubular Elastic Dressing Retainer, S...",DRESSING SURGILAST SZ 5,GL-705,GL705,INTEGRA LIFESCIENCES,INTEGRA LIFESCIENCES SALES LLC,42311543.0000,42311543,0,100,93,1,1,02 Med,Yes,NaN,NaN,1


#### Train and Test Split

In [90]:
raw_trn_df, raw_tst_df = train_test_split(feat_df, test_size=0.2, random_state=42)

In [91]:
raw_trn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1668 entries, 1866 to 860
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ITEMNUMBER              1668 non-null   object 
 1   Number                  1668 non-null   object 
 2   ITEMDESCRIPTION         1668 non-null   object 
 3   FullDescription         1668 non-null   object 
 4   SUPPLIER_ITEM_NUMBER    1668 non-null   object 
 5   ManufacturerItemNumber  1668 non-null   object 
 6   SUPPLIERNAME            1668 non-null   object 
 7   ManufacturerName        1668 non-null   object 
 8   UNSPCCode               1646 non-null   float64
 9   UNSPSC                  1668 non-null   int64  
 10  Feat_1                  1668 non-null   int64  
 11  Feat_2                  1668 non-null   int64  
 12  Feat_3                  1668 non-null   int64  
 13  Feat_4                  1668 non-null   int64  
 14  Feat_5                  1668 non-null 

In [92]:
raw_trn_df.Match.value_counts()

No     1015
Yes     653
Name: Match, dtype: int64

In [93]:
raw_tst_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 417 entries, 1403 to 303
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ITEMNUMBER              417 non-null    object 
 1   Number                  417 non-null    object 
 2   ITEMDESCRIPTION         417 non-null    object 
 3   FullDescription         417 non-null    object 
 4   SUPPLIER_ITEM_NUMBER    417 non-null    object 
 5   ManufacturerItemNumber  417 non-null    object 
 6   SUPPLIERNAME            417 non-null    object 
 7   ManufacturerName        417 non-null    object 
 8   UNSPCCode               411 non-null    float64
 9   UNSPSC                  417 non-null    int64  
 10  Feat_1                  417 non-null    int64  
 11  Feat_2                  417 non-null    int64  
 12  Feat_3                  417 non-null    int64  
 13  Feat_4                  417 non-null    int64  
 14  Feat_5                  417 non-null   

In [94]:
raw_tst_df.Match.value_counts()

No     261
Yes    156
Name: Match, dtype: int64

## Save Data to Modeling Environment

In [97]:
raw_trn_df.reset_index(drop=True).to_pickle(oput_dir + trn_oput_file_name)

In [98]:
raw_tst_df.reset_index(drop=True).to_pickle(oput_dir + tst_oput_file_name)